<h1><center>Лабораторна робота 5.</center></h1>
<h2><center>Логістична регресія й випадковий ліс в задачі кредитного скорингу</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

<a class="anchor" id="lab-5"></a>

## Зміст 

- [5.1. Завантаження навчальних даних](#lab-5.1)
- [5.2. Бутстреп](#lab-5.2)  
- [5.3. Підбір параметрів для моделі логістичної регресії](#lab-5.3)
- [5.4. Визначення впливу ознак](#lab-5.4)
- [5.5. Створення моделі випадкового лісу](#lab-5.5)
- [5.6. Беггінг](#lab-5.6)

Головним завданням цієї лабораторної роботи є побудова моделі на основі машинного навчання (МН) для задачі кредитного скорингу.

<a class="anchor" id="lab-5.1"></a>

## <span style="color:blue; font-size:1.2em;">5.1. Завантаження навчальних даних</span>

[Повернутися до змісту](#lab-5)

Змінна, значення якої прогнозується, $Y$:

 - SeriousDlqin2yrs – стан виплати людиною кредиту впроводж наступних 90 днів; можливі значення – 1 або 0. 

Незалежні ознаки, $X$:

 - age – вік позичальника кредитних коштів; тип – integer;
 - NumberOfTime30-59DaysPastDueNotWorse – кількість прострочених виплат інших кредитів більше 30-59 днів тому, але не більше впродовж останніх двох років; тип – integer;
 - DebtRatio – відношення щомісячного відрахування щодо заборгованості (кредити, аліменти тощо) до сукупного місячного доходу (percentage); тип – real;
 - MonthlyIncome – місячний дохід в доларах; тип – real;
 - NumberOfTimes90DaysLate – кількість прострочених виплат інших кредитів більше 90 днів; тип – integer;
 - NumberOfTime60-89DaysPastDueNotWorse – кількість прострочених виплат інших кредитів більш 60-89 днів, але не більше впродовж останніх двох років; тип – integer;
 - NumberOfDependents – кількість членів сім'ї позичальника; тип – integer.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# відключимо попередження Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

Створимо функцію, яка буде замінювати NaN значення на медіану в кожному стовпчику таблиці:

In [ ]:
def delete_nan(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table   

Завантажуємо дані для подальшого навчання:

In [ ]:
credit_scoring_url = 'https://raw.githubusercontent.com/radiukpavlo/Intelligent-data-analysis/main/02_assignments/ida_lab-05_logit-rf-credit-scoring/credit_scoring_sample.csv'

data = pd.read_csv(credit_scoring_url, sep =';')
data.head()

Розглянемо типи завантажених даних:

In [ ]:
data.dtypes

Переглянемо розподіл класів у цільовій змінні:

In [ ]:
ax = data['SeriousDlqin2yrs'].hist(orientation='horizontal', color='red')
ax.set_xlabel("number_of_observations")
ax.set_ylabel("unique_value")
ax.set_title("Target distribution")

print('Distribution of target')
data['SeriousDlqin2yrs'].value_counts()/data.shape[0]

Далі виберемо назви всіх ознак з таблиці, крім прогнозованого.

In [ ]:
independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

Застосовуємо функцію, яка замінює всі NaN значення на медіанне значення відповідного стовпця.

In [ ]:
table = delete_nan(data)

Розділяємо підготовлений набір даних на незалежні ознаки ($X$) та цільову ознаку ($y$).

In [ ]:
X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']

<a class="anchor" id="lab-5.2"></a>

## <span style="color:blue; font-size:1.2em;">5.2. Бутстреп</span>

[Повернутися до змісту](#lab-5)

Спершу обрахуйте інтервальну оцінку середнього віку (age) для надійних клієнтів та тих клієнтів, що прострочили виплату кредиту з "впевненістю" >= 90% . Використайте приклад з лекції, поставте `np.random.seed (0)`, як це зроблено в огляді.

In [ ]:
# Функція для генерації підвибірок за допомогою бутстрепу
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [ ]:
# Функція для інтервальної оцінки
def stat_intervals(stat, alpha):
    # Ваш код тут
    return pass

In [ ]:
# Зберігаємо дані щодо клієнтів в окремі numpy масиви 
churn_clients = # Ваш код тут
reliable_clients = # Ваш код тут

# Cтавимо seed для відтворюваності результатів
np.random.seed(0)

# Формуємо вибірки за допомогою бутстрепу і відразу обраховуємо за кожною з них середнє
churn_mean_scores = # Ваш код тут
reliable_mean_scores = # Ваш код тут

# Виводимо інтервальну оцінку середнього
print("Інтервальна оцінка середнього віку {0} клієнтів: {1}",  # Ваш код тут)

### <span style="color:red; font-size:1.5em;">Завдання 1</span>


**Для варіантів 1-5:**

Обрахуйте інтервальну оцінку середнього віку надійних клієнтів з параметрами `n_samples=1000` та `alpha=0.1`. 

**Для варіантів 6-10:**

Обрахуйте інтервальну оцінку середнього віку боржників з параметрами `n_samples=1000` та `alpha=0.05`. 

**Для варіантів 11-15:**

Обрахуйте інтервальну оцінку середнього віку боржників з параметрами `n_samples=1000` та `alpha=0.05`. 

**Для варіантів 16-20:**

Обрахуйте інтервальну оцінку середнього віку надійних клієнтів з параметрами `n_samples=500` та `alpha=0.1`. 

<a class="anchor" id="lab-5.3"></a>

## <span style="color:blue; font-size:1.2em;">5.3. Підбір параметрів для моделі логістичної регресії</span>

[Повернутися до змісту](#lab-5)

Однією з важливих метрик якості моделі є значення площі під [ROC-кривої](https://uk.wikipedia.org/wiki/ROC-%D0%BA%D1%80%D0%B8%D0%B2%D0%B0) (AUC). Значення ROC-AUC лежить від 0 до 1. Чим ближче значення метрики ROC-AUC до 1, тим якісніше відбувається класифікація моделлю.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

Використовуємо модуль [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) для побудови логістичної регресії. Через незбалансованість класів в цільовій функції додаємо параметр балансування. Також додамо параметр `random_state=5` для відтворюваності результатів.

In [ ]:
lr = LogisticRegression(random_state=5, class_weight= 'balanced')

Тепер спробуємо підібрати найкращий коефіцієнт регуляризації (коефіцієнт C в логістичній регресії) для моделі логістичної регрессії. Найкраще значення параметра С забезпечить нам оптимальність моделі, яка буде добре прогнозувати значення цільової функції і водночас не буде перенавчатися. Інші параметри залишаємо за замовчуванням.

In [ ]:
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10)}

Щоби підібрати коефіцієнт регуляризації переглянемо значення ROC-AUC на Stratified крос-валідації з 5 фолдів для кожного можливого значення коефіцієнта регуляризації за допомогою функції StratifiedKFold.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

Використайте [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) з метрикою ROC-AUC за параметром C.

In [ ]:
%%time
lr_grid_search = # Ваш код тут
lr_grid_search = lr_grid_search.fit(X, y)

### <span style="color:red; font-size:1.5em;">Завдання 2</span>


**Для варіантів 1-5:**

Яке оптимальне значення параметра С в результаті застосування `GridSearchCV()`?

**Для варіантів 6-10:**

Виведіть значення стандартного відхилення за результатами `GridSearchCV()`.

**Для варіантів 11-15:**

Чи можна вважати кращу модель стійкою? Модель вважаємо стійкою, якщо стандартне відхилення на валідації менше 0.5% (скористайтесь результатми застосування `GridSearchCV()`).

**Для варіантів 16-20:**

Виведіть значення accuracy для оптимального значення параметра С за результатами `GridSearchCV()`. 

***Примітка:*** для розв'язання завдання 2 скористайтесь методоми `grid_search.best_estimator_`, `grid_search.cv_results_`, `grid_search.best_score_`.

**Збережіть точність кращої моделі, вона вам доводиться для наступних завдань.**

<a class="anchor" id="lab-5.4"></a>

## <span style="color:blue; font-size:1.2em;">5.4. Визначення впливу ознак</span>

[Повернутися до змісту](#lab-5)

У цьому завданні ми попрацюємо з важливістю ознак. Пам'ятаємо, що важливість ознаки визначається **абсолютним значенням її коефіцієнта**. Крім того, потрібно заздалегідь нормалізувати всі ознаки, щоб їх правильно порівняти.

In [ ]:
from sklearn.preprocessing import StandardScaler

lr = LogisticRegression(C=0.001,random_state=5, class_weight='balanced')
scal = StandardScaler()
lr.fit(scal.fit_transform(X), y)

Обрахуємо важливість ознак:

In [ ]:
pd.DataFrame({'feat': independent_columns_names,
              'coef': lr.coef_.flatten().tolist()}).sort_values(by='coef', ascending=False)

### <span style="color:red; font-size:1.5em;">Завдання 3</span>


**Для варіантів 1-5:**

Обрахуйте частку впливу найменш важливої ознаки на прогноз. Для цього закодуйте та обчисліть функцію [softmax](https://en.wikipedia.org/wiki/Softmax_function).

**Для варіантів 6-10:**

Обрахуйте частку впливу найважливішої ознаки на прогноз. Для цього закодуйте та обчисліть функцію [softmax](https://en.wikipedia.org/wiki/Softmax_function).

**Для варіантів 11-15:**

Порахуйте, у скільки разів збільшаться шанси, що клієнт не виплатить кредит, якщо збільшити вік на 10 років при всіх інших рівних значеннях ознак (теоретичний розрахунок можна подивитися [тут](https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf)).

**Для варіантів 16-20:**

Порахуйте, у скільки разів збільшаться шанси, що клієнт не виплатить кредит, якщо збільшити кількість членів сім'ї позичальника на 2 людини при всіх інших рівних значеннях ознак (теоретичний розрахунок можна подивитися [тут](https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf)).

In [ ]:
# Ваш код тут

***Примітка 1:*** Приклад реалізації функції softmax можете підчитати [тут](https://machinelearningmastery.com/softmax-activation-function-with-python/).

***Примітка 2:***

Формула, за якою обраховують у скільки разів збільшаться шанси при підвищенні знчення ознаки:

$$ \large \exp^{\beta\delta}, $$

де

$ \beta $ – значення ознаки; $ \delta $ – на скільки робимо приріст.



<a class="anchor" id="lab-5.5"></a>

## <span style="color:blue; font-size:1.2em;">5.5. Створення моделі випадкового лісу</span>

[Повернутися до змісту](#lab-5)

Ініціалізуємо випадковий ліс з 100 деревами й збалансованими класами:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, 
                            random_state=42, oob_score=True, 
                            class_weight='balanced')

Будемо шукати кращі параметри поміж набору:

In [ ]:
parameters = {'max_features': [1, 2, 4], 
              'min_samples_leaf': [3, 5, 7, 9], 
              'max_depth': [5, 10, 15]}

Робимо знову Stratified k-fold валідацію. Ініціалізація валідації має зберігатися в skf з попередніх завдань.

In [ ]:
%%time
rf_grid_search = # Ваш код тут

rf_grid_search = rf_grid_search.fit(X, y)

### <span style="color:red; font-size:1.5em;">Завдання 4</span>


**Для варіантів 1-5:**

Відповідно до результатів класифікації за випадковим лісом, яка ознака має найбільший вплив на цільову функцію? Виведіть назву ознаки та числове значення її впливовості.

**Для варіантів 6-10:**

Порівняйте результати класифікації за логістичною регресією та випадковим лісом. Вкажіть переваги та недоліки обох підходів до розв'язування нашої задачі кредитного скорингу.

Для висновку використайте такі тези:
- менше витрачається часу на навчання моделі;
- менше параметрів для перебору;
- хороше інтерпретування ознак;
- лінійні властивості алгоритму.

**Для варіантів 11-15:**

Відповідно до результатів класифікації за випадковим лісом, яка ознака має найменший вплив на цільову функцію? Виведіть назву ознаки та числове значення її впливовості.

**Для варіантів 16-20:**

Порівняйте значення частки правильних відповідей (ЧПВ, accuracy) кращої моделі випадкового лісу з кращою моделюю логістичної регресії на валідації?

<a class="anchor" id="lab-5.6"></a>

## <span style="color:blue; font-size:1.2em;">5.6. Беггінг</span>

[Повернутися до змісту](#lab-5)

Наступне завдання полягатиме у навчанні беггінга, як класифікатора (`random_state=42`).

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV

За базові класифікатори візьміть 100 логістичних регресій. Виконайте тюнинг моделі з використанням техніки `RandomizedSearchCV`. Значення параметрів для перебору дано нижче:

In [ ]:
parameters = {'max_features': [1, 2, 3, 4, 5], 'max_samples': [0.1, 0.3, 0.5, 0.7, 0.9], 
              "base_estimator__C": [0.0001, 0.001, 0.01, 1, 10, 100]}

In [ ]:
bg = BaggingClassifier(# Ваш код тут
                        )

Позаяк перебирати всі варіанти комбінацій надто довго, поставте максимальне число ітерацій 20 для [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Також не забудьте передати параметр валідації `CV` і `random_state=1`.

In [ ]:
%%time
bg_random_search = # Ваш код тут

bg_random_search = bg_random_search.fit(X, y)

### <span style="color:red; font-size:1.5em;">Завдання 5</span>


**Для варіантів 1-10:**

Виконайте додактово тюнинг навченого беггінга за допомогою `GridSearchCV`. Порівняйте результати тюнингу за `RandomizedSearchCV` та `GridSearchCV`.

**Для варіантів 11-20:**

Яку найкращу частку правильних відповідей отримано за допомогою `RandomizedSearchCV`?

Дайте інтерпретацію кращих параметрів для беггінга. Чому саме такі значення виявилися кращими?